### Download dataset using SPARQL

In [10]:
import pandas as pd
from IPython.core.display import display
from IPython.display import Markdown, clear_output
from ipywidgets import Layout, Box, VBox, widgets

ENDPOINT: str = "http://kaiko.getalp.org/sparql" # "https://statistics.gov.scot/sparql"

In [11]:
%run ../Dataset_tools

In [12]:
list_datasets = get_datasets(ENDPOINT, verbose=False)  # List of all datasets available
list_datasets_short = pd.concat(
    {'dataset': list_datasets['dataset'].map(lambda x: x.split('/')[-1]), 'commentaire': list_datasets['commentaire']})

DataFrame = pd.DataFrame()  # The dataframe that will contain the dataset
output = widgets.Output()

if len(list_datasets) == 0:
    print("No data set was found")
else:
    dataset_user_choice = widgets.Dropdown(
        options=[(name, full_name) for name, full_name in
                 zip(list_datasets_short['dataset'].values, list_datasets['dataset'].values)],
        description="Choice:", layout=Layout(flex='1 3 auto', width='auto')
    )  # The widget that will allow the user to choose which dataset to download

    user_choice_confirm = widgets.Button(description='Submit', icon='check',
                                         layout=Layout(flex='1 1 auto', width='auto'),
                                         tooltip="Click here to confirm your choice"
                                         )  # The widget that will allow the user to confirm his choice of dataset

    description = widgets.Label(
        value=f"Description: {list_datasets[list_datasets['dataset'] == dataset_user_choice.value]['commentaire'].values[0]}",
        layout=Layout(flex='1 1 auto',
                      width='auto'))  # The widget that will display the description of the selected dataset if it exists

    box_layout = Layout(display='flex', flex_flow='row', align_items='stretch', width='90%')

    line_1 = Box(children=[dataset_user_choice, user_choice_confirm], layout=box_layout)
    line_2 = Box(children=[description], layout=box_layout)
    ui = VBox([line_1, line_2])


    def user_choice_change(obj):  # Update the value of label widget when the user_choice widget change of value
        description.value = f"Description: {list_datasets[list_datasets['dataset'] == dataset_user_choice.value]['commentaire'].values[0]}"


    def user_choice_confirm_eventhandler(obj):
        with output:
            clear_output()
            output2 = widgets.Output()

            categories_long = get_features(ENDPOINT, dataset_user_choice.value)  # Get all features of choosed dataset
            categories_short = categories_long["property"].map(lambda x: x.split('#')[-1].split('/')[-1]).to_frame()

            display(Markdown(data="#### The features of this dataset are: "), categories_short)

            # The widget that will allow the user to choose which features download
            select = widgets.SelectMultiple(
                options=[(name, full_name) for name, full_name in zip(categories_short.values.reshape(-1),
                                                                      categories_long.values.reshape(-1))],
                description='Critères: ',
                layout=Layout(flex='1 1 auto', width='auto', height="auto"), style={'description_width': 'initial'})

            # The widget that will allow the user to confirm his choice of features
            select_confirm = widgets.Button(description='Soumettre', icon='check',
                                            layout=Layout(flex='1 1 auto', width='auto'),
                                            tooltip="Cliquer ici pour confirmer votre choix")

            select_box = Box(children=[select], layout=box_layout)
            select_confirm_box = Box(children=[select_confirm], layout=box_layout)
            select_ui = VBox([select_box, select_confirm_box])

            def selection_confirm_eventhandler(obj):
                with output2:
                    if len(select.value) >= 2:  # We will constuct the query
                        global DataFrame
                        clear_output()
                        # Download selected features from the specified dataset
                        DataFrame = download_dataset(ENDPOINT, dataset_user_choice.value, select.value)
                        display(DataFrame)  # Display fists and lasts rows of the dataframe

            select_confirm.on_click(selection_confirm_eventhandler)
            display(Markdown("#### Please choose two features at least: "), select_ui, output2)


    user_choice_confirm.on_click(user_choice_confirm_eventhandler)
    dataset_user_choice.observe(user_choice_change, 'value')
    display(Markdown(data="#### Please choose a dataset: "), ui, output)

IntProgress(value=0, bar_style='success', description='Loading:')

#### Please choose a dataset: 

Output()